# House Prices


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from importlib import reload
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
import optuna
import pickle


from modules import function


In [18]:
# Get raw data
train_data_raw = pd.read_csv("../data/raw/train.csv")
test_data_raw = pd.read_csv("../data/raw/test.csv")

train_data_raw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [16]:
# numerical info
train_data_raw.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [19]:
# object data
train_data_raw.describe(include="O")

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,...,1379,1379,1379,1379,1460,7,281,54,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,...,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,...,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,50,925,1311,1459,1052,1382,225,1260,...,870,605,1311,1326,1340,3,157,49,1267,1198


In [81]:
train_data_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [106]:
def separate_df(df):
    df_n=df.select_dtypes(include=['int64', 'float64'])
    df_o=df.select_dtypes(include=["O"])
    return df_n, df_o

_, train_data_o = separate_df(train_data_raw)
_, test_data_o = separate_df(test_data_raw)

# for (col1, series1), (col2, series2) in zip(train_data_o.items(), test_data_o.items()):
#     print(col1, len(set(series1)), len(set(series2)))

for col_name in train_data_o.columns:
    one_hot_train = pd.get_dummies(train_data_o[col_name], prefix=col_name)
    one_hot_test = pd.get_dummies(test_data_o[col_name], prefix=col_name)
    
    if (one_hot_train.shape[1] != one_hot_test.shape[1]):
        print(col_name)

Utilities
Condition2
HouseStyle
RoofMatl
Exterior1st
Exterior2nd
Heating
Electrical
GarageQual
PoolQC
MiscFeature


In [87]:
#　訓練とテストでスパースな列を抽出
threshold = 0.5

sparse_col = []
for data in [train_data_raw, test_data_raw]:
    for col_name, series in data.items():
        num_nan = series.isna().sum()
        if (num_nan > len(data) * threshold):
            sparse_col.append(col_name)

sparse_col = list(set(sparse_col))

In [108]:
def preprocess(df):
    
    #　欠損値の多い列を削除
    df = df.drop(columns=sparse_col)
    df.pop("Id")
    
    #　数値データとオブジェクトデータを分ける
    df_n=df.select_dtypes(include=['int64', 'float64'])
    df_o=df.select_dtypes(include=["O"])
    assert df.shape[1] == df_n.shape[1] + df_o.shape[1], "列の数が一致していない"
    
    # one hot encoding
    for col_name in df_o.columns:
        df_o = function.add_one_hot(df_o, col_name)
    
    df = pd.concat([df_n, df_o], axis=1)

    return df

train_data = preprocess(train_data_raw)
test_data = preprocess(test_data_raw)



y_train = train_data["SalePrice"]
x_train = train_data.drop(columns="SalePrice")
x_test = test_data

# 同じ列名のみ採用
columns_to_keep = list(set(x_train.columns) & set(x_test.columns))
x_train = x_train[columns_to_keep]
x_test = x_test[columns_to_keep]

assert x_train.shape[1] == x_test.shape[1], "列の数が違います"


In [119]:
lgbm_params = {
        'boosting_type': 'gbdt',
        'num_leaves': 10,
        "max_depth": 5,
        'learning_rate': 0.01,
        'n_estimators': 200,
        'random_state': 42,
    }

clf = lgb.LGBMRegressor(**lgbm_params)
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

# Save prediction
Id = np.array(test_data_raw["Id"]).astype(int)
function.save_kaggle_prediction(Id, y_pred, "Id", "SalePrice")


# パラメータチューニング

In [ ]:
n_splits = 5
#　クロスバリデーション用
cv_cv = StratifiedKFold(
        n_splits=n_splits, shuffle=True, random_state=42)
#　最適化用
cv_opt = StratifiedKFold(
        n_splits=3, shuffle=True, random_state=42)
#　探索範囲
params={"num_leaves":[2,30],"learning_rate":[0.01,0.2],"max_depth":[1,10]}
study_list = function.cv_lgbm_bayesian_opt(x_train, y_train, cv_cv, cv_opt, params, n_trials=100)

with open("../data/processed/study_list_"+function.get_now_time()+".bin", 'wb') as p:
    pickle.dump(study_list, p)


# モデル学習

In [55]:
with open("../data/processed/study_list_20230525_154947.bin", 'rb') as p:
    study_list = pickle.load(p)

best_pramas_list = []
for study in study_list:
    best_pramas_list.append(study.best_params)
    
#学習   
clf_list = function.cv_lgbm_clf(x_train, y_train, cv_cv, best_pramas_list, eval_metric="logloss", stopping_rounds=50)
with open("../models/clf_list_"+function.get_now_time()+".bin", 'wb') as p:
    pickle.dump(clf_list, p)

# 予測

In [56]:
# 多数決
with open("../models/clf_list_20230526_155950.bin", 'rb') as p:
    clf_list = pickle.load(p)

# 3以上なら1とする
sum_pred = np.zeros(len(x_test))
for clf in clf_list:
    sum_pred+=clf.predict(x_test)
y_pred = np.where(sum_pred >= 3, 1, 0)

# Save prediction
PassengerId = np.array(test_data_raw["PassengerId"]).astype(int)
function.save_kaggle_prediction(y_pred, PassengerId, "Survived", "PassengerId")

In [ ]:


x_label = "Age"
y_label = "Fare"
fig, ax = plt.subplots(figsize=(10,6))
scatter = ax.scatter(train_data_raw[x_label], train_data_raw[y_label], c=train_data_raw["Survived"], cmap='viridis')

# 軸ラベルの設定
ax.set_xlabel(x_label)
ax.set_ylabel(y_label)

# 凡例の表示
legend_labels = ['Dead', 'Survived']
ax.legend(handles=scatter.legend_elements()[0], labels=legend_labels, title='Labels')
